#**AG3- Actividad Guiada 3**

Nombre: Mikel Pérez Frutos <br>
Link: https://colab.research.google.com/drive/1iYNWQ4q1SBSHSyKcJwLq65znEXVzKHvx?usp=sharing <br>
Github: https://github.com/mikelinnho/03MIAR---Algoritmos-de-Optimizacion


#**Librerias**
Carga de las liberías:

In [ ]:
!pip install requests #Hacer llamadas http a paginas de la red
!pip install tsplib95 #Módulo para las instancias del problema del TSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#**Carga de datos**
Carga de los datos del problema:

In [ ]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95 #Módulo para las instancias del problema del TSP
import math #Módulo de funciones matematicas. Se usa para exp
import random #Para generar valores aleatorios

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html (Todas las funciones)
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz #Descomprimir el fichero de datos 

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/eil51.tsp.gz", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/att48.tsp.gz", file)

gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? y


Carga de datos en las variables del problema:

In [ ]:
problema = tsplib95.load(file)
nodos = list(problema.get_nodes()) #Nodos
aristas = list(problema.get_edges()) #Aristas

Algunas funciones del objeto "problema":

In [ ]:
problema.get_weight(0, 1)

15

#**Funciones básicas**


Algunas funciones generales:

In [ ]:
def crear_solucion(nodos): 
  #Definición: Se genera una solucion aleatoria con comienzo en en el nodo 0
  #Parámetros:
  # - nodos conjunto de elementos.
  #Return: Solución comenzando en el nodo 0

  solucion = [nodos[0]]
  for n in nodos[1:]:
    solucion = solucion + [random.choice(list(set(nodos) - set({nodos[0]}) - set(solucion)))]
  return solucion 

def distancia(a, b, problema):
  #Definición: Devuelve la distancia entre dos nodos
  #Parámetros:
  # - a y b son identificadores de nodos.
  # - problema es la matriz de distancias.
  #Return: Distancia entre dos nodos

  return problema.get_weight(a,b)

def distancia_total(solucion, problema):
  #Definición: Devuelve la distancia total de una trayectoria/solucion
  #Parámetros:
  # - solucion es la trayectoria.
  # - problema es la matriz de distancias.
  #Return: Distancia total de una trayectoria

  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i], solucion[i+1], problema)
  return distancia_total + distancia(solucion[len(solucion)-1], solucion[0], problema)

#**Búsqueda aleatoria**
**Definición**: Es un proceso por el que se van generando soluciones aleatorias en cada iteración y
se devuelve la mejor. 

In [ ]:
###################################
#       Búsqueda aleatoria        #
###################################
def busqueda_aleatoria(problema, N):
  #Definición: Calcula la menor distancia para una trayectoria.
  #Parámetros:
  # - problema es la matriz de distancias.
  # - N es el numero de iteraciones
  #Return: Resultado de la búsqueda aleatoria

  nodos = list(problema.get_nodes())
  mejorSolucion = []
  mejorDistancia = float('inf') #Inicializamos con un valor alto
  
  for i in range(N): #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(nodos) #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problema) #Calcula el valor objetivo(distancia total)  
    if distancia < mejorDistancia: #Compara con la mejor obtenida hasta ahora
      mejorSolucion = solucion
      mejorDistancia = distancia
      
  print("Mejor solución: ", mejorSolucion) 
  print("Distancia: ", mejorDistancia) 
  return mejorSolucion  

Buúsqueda aleatoria con 5000 iteraciones:

In [ ]:
solucion = busqueda_aleatoria(problema, 5000)    

Mejor solución:  [0, 23, 25, 40, 41, 8, 38, 3, 29, 28, 4, 15, 17, 36, 7, 12, 11, 19, 6, 37, 35, 13, 27, 5, 26, 2, 1, 31, 20, 30, 21, 14, 10, 18, 24, 34, 33, 39, 22, 32, 9, 16]
Distancia:  3790


#**Búsqueda local**

In [ ]:
###################################
#         Búsqueda local          #
###################################
def genera_vecina(solucion, problema):
  #Definición: Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Parámetros:
  # - problema es la matriz de distancias.
  # - N es el numero de iteraciones
  #Return: Solución vecina

  mejorSolucion = []
  mejorDistancia = float('inf') #Inicializamos con un valor alto

  for i in range(1, len(solucion)-1): #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):
      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #Usamos el operador + que para listas en python las concatena): ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]   

      #Se evalua la nueva solución:
      distancia_vecina = distancia_total(vecina, problema)         

      #Para guardarla si mejora las anteriores
      if distancia_vecina <= mejorDistancia:
        mejorDistancia = distancia_vecina
        mejorSolucion = vecina

  return mejorSolucion

def busqueda_local(problema, nodos):
  #Definición: Calcula la menor distancia para una trayectoria.
  #Parámetros:
  # - problema es la matriz de distancias.
  # - nodos conjunto de elementos.
  #Return: Resultado de la búsqueda aleatoria
  mejor_solucion = []
  
  #Generar una solucion inicial de referencia(aleatoria):
  solucion_referencia = crear_solucion(nodos)
  mejor_distancia = distancia_total(solucion_referencia, problema)
  iteracion = 0 #Contador para saber las iteraciones que hacemos

  while(1):
    #Incrementamos el contador:
    iteracion +=1         

    #Obtenemos la mejor vecina:
    vecina = genera_vecina(solucion_referencia, problema)

    #Evaluamos la vecina para ver si mejoramos respecto a lo encontrado hasta el momento:
    distancia_vecina = distancia_total(vecina, problema)
 
    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina #Guarda la mejor distancia encontrada
    else:
      print("En la iteracion", iteracion, "la mejor solución encontrada es:" , mejor_solucion) 
      print("Distancia:" , mejor_distancia) 
      return mejor_solucion
      
    solucion_referencia = vecina

Generando soluciones vecinas:

In [ ]:
print("Distancia solución inicial:" , distancia_total(solucion, problema))
nueva_solucion = genera_vecina(solucion, problema)
print("Distancia mejor solución local:", distancia_total(nueva_solucion, problema))

Distancia solución inicial: 3790
Distancia mejor solución local: 3358


Y con una búsqueda local:

In [ ]:
sol = busqueda_local(problema, nodos)

En la iteracion 33 la mejor solución encontrada es: [0, 38, 22, 39, 9, 23, 41, 26, 5, 13, 19, 32, 34, 33, 20, 35, 36, 31, 1, 6, 7, 17, 37, 15, 16, 14, 21, 40, 24, 30, 28, 2, 27, 3, 4, 29, 8, 10, 25, 11, 12, 18]
Distancia: 1922


#**Recocido simulado**




In [ ]:
###################################
#      Simulated annealing        #
###################################
def genera_vecina_aleatorio(solucion):
  #Definición: Generador de soluciones vecinas
  #Parámetros:
  # - solucion es la trayectoria.
  #Return: Solución vecina

  #Se eligen dos nodos aleatoriamente:
  i,j = sorted(random.sample(range(1,len(solucion)) , 2))
  
  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar:
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  
def probabilidad(T, d):
  #Definición: Funcion de probabilidad para aceptar peores soluciones
  #Parámetros:
  # - T es la temperatura
  # - d es la distancia
  #Return: True si se acepta, False si no

  if random.random() < math.exp(-1*d / T):
    return True
  else:
    return False

def bajar_temperatura(T):
  #Definición: Funcion de descenso de temperatura
  #Parámetros:
  # - T es la temperatura
  #Return: temperatura multiplicada por 0,99

  return T*0.99

def recocido_simulado(problema, T, nodos):
  #Definición: Funcion de aplica simulated annealing para calcular la trayectoria
  #Parámetros:
  # - T es la temperatura
  # - problema es la matriz de distancias.
  # - nodos conjunto de elementos.
  #Return: Mejor trayectoria
  
  solucion_referencia = crear_solucion(nodos)
  distancia_referencia = distancia_total(solucion_referencia, problema)
  mejor_solucion = []  
  mejor_distancia = 10e100

  while T > .0001:
    #Genera una solución vecina:
    vecina = genera_vecina_aleatorio(solucion_referencia)
    
    #Calcula su valor(distancia):
    distancia_vecina = distancia_total(vecina, problema)
      
    #Si es la mejor solución de todas se guarda:
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    #Si la nueva vecina es mejor se cambia  
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(T, abs(distancia_referencia - distancia_vecina)):
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura:
    T = bajar_temperatura(T)
 
  print("La mejor solución encontrada es", mejor_solucion)
  print("Con una distancia total de", mejor_distancia)
  return mejor_solucion

La solución queda de la siguiente forma:

In [ ]:
sol  = recocido_simulado(problema, 10000000, nodos)

La mejor solución encontrada es [0, 3, 28, 30, 38, 22, 24, 40, 21, 8, 26, 16, 37, 36, 35, 17, 31, 32, 27, 41, 23, 9, 39, 29, 2, 4, 5, 19, 13, 18, 12, 11, 25, 10, 34, 33, 20, 7, 1, 6, 15, 14]
Con una distancia total de 2015


## **Practica individual**
Tareas opcionales para mejorar nota:

##**Búsqueda local con entornos variables**
*¿Se puede mejorar con otros operadores de vecindad variables?* <br>

Se va a tratar de modificar cómo se generan nuevas soluciones, para evitar los mínimos locales. Para ello, sobre el algoritmo generador de vecindad:

* Se va a invertir la lista generada.
* Se va a ordenar la lista generada creciente y decrecientemente.
* Se va a aleatorizar la lista generada.

Para ordenar la lista, vamos a implementar el algoritmo de ordenación MergeSort.

In [ ]:
def merge_sort(arr):
  if len(arr) == 1: return arr
  mid = len(arr) // 2
  return marge(merge_sort(arr[:mid]), merge_sort(arr[mid:]))

def marge(left, right):
  result = []
  while len(left) > 0 and len(right) > 0:
    if left[0] <= right[0]:
      result.append(left.pop(0))
    else:
      result.append(right.pop(0))

  result += left
  result += right
  return result

def genera_vecina_new(solucion, problema):
  #Definición: Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Parámetros:
  # - problema es la matriz de distancias.
  # - N es el numero de iteraciones
  #Return: Solución vecina

  mejor_solucion = []
  mejor_distancia = 10e100

  for i in range(1, len(solucion)-1): #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):
      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #Usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Modificaciones sobre el generador de vecindad:
      vecina_inv = list(reversed(vecina.copy()))
      vecina_ord = (merge_sort(vecina.copy()))
      vecina_ord_inv = list(reversed(vecina_ord.copy()))
      vecina_random = random.sample(vecina.copy(), len(vecina))

      #Se evalua la nueva solución:
      distancia_vecina = distancia_total(vecina, problema)
      distancia_vecina_inv = distancia_total(vecina_inv, problema)
      distancia_vecina_ord = distancia_total(vecina_ord, problema)
      distancia_vecina_ord_inv = distancia_total(vecina_ord_inv, problema)
      distancia_vecina_random = distancia_total(vecina_random, problema)

      #Guardar la mejor solución entre todas:
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
      if distancia_vecina_inv <= mejor_distancia:
        mejor_distancia = distancia_vecina_inv
        mejor_solucion = vecina_inv
      if distancia_vecina_ord <= mejor_distancia:
        mejor_distancia = distancia_vecina_ord
        mejor_solucion = vecina_ord
      if distancia_vecina_ord_inv <= mejor_distancia:
        mejor_distancia = distancia_vecina_ord_inv
        mejor_solucion = vecina_ord_inv
      if distancia_vecina_random <= mejor_distancia:
        mejor_distancia = distancia_vecina_random
        mejor_solucion = vecina_random

  return mejor_solucion

def busqueda_local(problema, nodos):
  #Definición: Calcula la menor distancia para una trayectoria.
  #Parámetros:
  # - problema es la matriz de distancias.
  # - nodos conjunto de elementos.
  #Return: Resultado de la búsqueda aleatoria
  mejor_solucion = []
  
  #Generar una solucion inicial de referencia(aleatoria):
  solucion_referencia = crear_solucion(nodos)
  mejor_distancia = distancia_total(solucion_referencia, problema)
  iteracion = 0 #Contador para saber las iteraciones que hacemos

  while(1):
    #Incrementamos el contador:
    iteracion +=1         

    #Obtenemos la mejor vecina:
    vecina = genera_vecina_new(solucion_referencia, problema)

    #Evaluamos la vecina para ver si mejoramos respecto a lo encontrado hasta el momento:
    distancia_vecina = distancia_total(vecina, problema)
 
    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina #Guarda la mejor distancia encontrada
    else:
      print("En la iteracion", iteracion, "la mejor solución encontrada es:" , mejor_solucion) 
      print("Distancia:" , mejor_distancia) 
      return mejor_solucion
      
    solucion_referencia = vecina

In [ ]:
print("Distancia solución inicial:" , distancia_total(solucion, problema))
nueva_solucion = genera_vecina_new(solucion, problema)
print("Distancia mejor solución local:", distancia_total(nueva_solucion, problema))

Distancia solución inicial: 3790
Distancia mejor solución local: 2834


Por tanto, la búsqueda local quedaría:

In [ ]:
sol = busqueda_local(problema, nodos)

En la iteracion 29 la mejor solución encontrada es: [21, 39, 2, 3, 4, 6, 5, 26, 18, 10, 25, 11, 12, 13, 19, 14, 16, 15, 37, 0, 30, 38, 22, 9, 23, 41, 8, 29, 28, 27, 32, 34, 33, 20, 31, 35, 36, 17, 7, 1, 40, 24]
Distancia: 1716


##**Recocido simulado**
*¿Se puede mejorar con otra elección no tan aleatoria (función genera_vecina_aleatorio())?*

Para mejorar el resultado del algoritmo de "Recocido Simulado", se han aplicado las modificaciones presentadas en el siguiente paper: https://www.hindawi.com/journals/cin/2016/1712630/ <br>

La mejora que se presenta consiste en añadir al algoritmo una estructura de
listas para controlar la disminución de la temperatura, de manera que se simplifique la configuración de parámetros. <br>

También, para generar nuevas soluciones vecinas, se establecen 3 nuevos operadores: 
* El operador de inversión.
* El operador de inserción.
* El operador de intercambio. 

En este caso, se va a implementar únicamente el operador de inversión. La diferencia entre este operador y el usado anteriormente, es que en lugar de intercambiar únicamente los nodos aleatorios i y j, se van a invertir todas las ciudades entre las posiciones i y j.

In [ ]:
def genera_vecina_aleatoria_new(solucion):
  #Definición: Generador de soluciones vecinas
  #Parámetros:
  # - solucion es la trayectoria.
  #Return: Solución vecina

  #Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))
  #Se invierten los caminos que unen i y j
  solucion_inv = list(reversed(solucion[i+1:j].copy()))
  # Devuelve una nueva solución pero intercambiando los caminos entre i y j, por su inversa
  return solucion[:i+1] + solucion_inv + solucion[j:]

def recocido_simulado_new(problema, T, nodos):
  #Definición: Funcion de aplica simulated annealing para calcular la trayectoria
  #Parámetros:
  # - T es la temperatura
  # - problema es la matriz de distancias.
  # - nodos conjunto de elementos.
  #Return: Mejor trayectoria
 
 solucion_referencia = crear_solucion(nodos)
 distancia_referencia = distancia_total(solucion_referencia, problema)
 mejor_solucion = []
 mejor_distancia = 10e100
 N = 0

 while T > .0001:
  N += 1
  #Genera una solución vecina:
  vecina = genera_vecina_aleatoria_new(solucion_referencia)

  #Calcula su valor(distancia):
  distancia_vecina = distancia_total(vecina, problema)

  #Si es la mejor solución de todas se guarda:
  if distancia_vecina < mejor_distancia:
    mejor_solucion = vecina
    mejor_distancia = distancia_vecina

  #Si la nueva vecina es mejor se cambia 
  #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
  if distancia_vecina < distancia_referencia or probabilidad(T, abs(distancia_referencia - distancia_vecina)):
    solucion_referencia = vecina
    distancia_referencia = distancia_vecina

  #Bajamos la temperatura
  T = bajar_temperatura(T)

 print("La mejor solución encontrada es", mejor_solucion)
 print("Distancia total de", mejor_distancia)
 return mejor_solucion

La solución queda de la siguiente forma:

In [ ]:
sol = recocido_simulado_new(problema, 100000000, nodos)

La mejor solución encontrada es [0, 23, 41, 10, 25, 11, 12, 18, 4, 3, 1, 7, 5, 26, 6, 13, 19, 14, 37, 15, 16, 36, 35, 31, 17, 32, 34, 20, 33, 38, 22, 30, 28, 27, 2, 29, 8, 9, 39, 21, 24, 40]
Distancia total de 1782
